# Big Data Modeling and Management Assigment - Homework 1

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.

### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions


### Questions

1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 5-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer well!) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [Hint: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer well!) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 13**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/HW1/Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/Neo4Jplugins`.
- Change the code bellow accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/Neo4Jplugins":/plugins -v "C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/HW1/Neo4JHWData/data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist=gds.* neo4j`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

In [1]:
from neo4j import GraphDatabase
from pprint import pprint

In [2]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

### All The Functions you'll need to run queries in Neo4J

In [4]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

### Understanding the Database

In [5]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [6]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


# Submission

GROUP NUMBER:

**Group25**

GROUP MEMBERS:

|STUDENT NUMBER|STUDENT NAME|
|---|---|
|20191224|André Cunha|
|20191211|Catarina Duarte|
|20191190|Dinis Melo|
|20220646|Fernando Cruz|
|20220592|Pedro Fernandes|

#### 1. How many different countries exist in the database?

In [7]:
query = """
        MATCH (c:COUNTRIES)
        RETURN DISTINCT count(*)
    """

result = execute_read(driver, query)

pprint(result)

[<Record count(*)=200>]


#### 2. Most reviews:  
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

In [8]:
query = """
        MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN b.name, count(r) AS nr_reviews       
        ORDER BY nr_reviews desc
        LIMIT 1
    """
#We decided to go with count(r) instead of count(*) to only count the reviews
#We returned the name of the beer with the most reviews as well as the respective reviews count

result = execute_read(driver, query)

pprint(result)

[<Record b.name='IPA' nr_reviews=31387>]


In [9]:
query = """
        MATCH (br:BREWERIES)-[bre:BREWED]->(b:BEERS)-[rev:REVIEWED]->(r:REVIEWS)
        RETURN br.name, count(r) AS nr_reviews
        ORDER BY nr_reviews desc
        LIMIT 1
    """

#We just added (br:BREWERIES)-[bre:BREWED]-> to the previous query to relate each beer to its 
#brewery and consequetly to the reviews

result = execute_read(driver, query)

pprint(result)

[<Record br.name='Sierra Nevada Brewing Co.' nr_reviews=175161>]


In [10]:
query = """
        MATCH (c:COUNTRIES)<-[:IN]-(cit:CITIES)<-[:IN]-(br:BREWERIES)-[bre:BREWED]->(b:BEERS)-[rev:REVIEWED]->(r:REVIEWS)
        RETURN c.name, count(r) AS nr_reviews
        ORDER BY nr_reviews desc
        limit 1
    """
#We just added (c:COUNTRIES)<-[:IN]-(cit:CITIES)<-[:IN]- to the previous query to relate each country and city to each 
#brewery and beer and consequently to the reviews

result = execute_read(driver, query)

pprint(result)

[<Record c.name='US' nr_reviews=7675804>]


#### 3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [11]:
query = """
        MATCH (u1:USER{name:"CTJman"})<-[:POSTED]-(r1:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:REVIEWED]->(r2:REVIEWS)-[:POSTED]->(u2:USER)
        RETURN u2.name, count(b) as nr_beers
        ORDER BY nr_beers DESC
        LIMIT 1
    """
#We started the match with the user we wanted (CTJman) and did the relations until beers
#That way, would have access to all the beers which were reviewed by CTJman
#Then, we connected beers to users (u2), so we would have access to all the users which reviewed some of the same beers
#When returning the u2 users names and the count of beers, we had access to the number of beers reviewed in common with CTJman

result = execute_read(driver, query)

pprint(result)

[<Record u2.name='acurtis' nr_beers=1428>]


#### 4. Which Portuguese brewery has the most beers?


In [12]:
query = """
        MATCH (c:COUNTRIES{name:"PT"})<-[:IN]-(cit:CITIES)<-[:IN]-(br:BREWERIES)-[bre:BREWED]->(b:BEERS)
        RETURN br.name, count(b) as nr_beers
        ORDER BY nr_beers desc
        limit 1
    """

#We connected the countries (specifically Portugal) to beers through cities and breweries and then returned the names of the breweries
#and the number of beers, that way we would know the number of beers each Portuguese brewery had

result = execute_read(driver, query)

pprint(result)

[<Record br.name='Dois Corvos Cervejeira' nr_beers=40>]


#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


In [13]:
query = """
        MATCH (br:BREWERIES{name:"Dois Corvos Cervejeira"})-[bre:BREWED]->(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN br.name, b.name, count(r) as nr_reviews
        ORDER BY nr_reviews desc
        LIMIT 1
    """

#This query was similar to the previous (4), since we only had to go from breweries (specifying the name of it) to reviews

result = execute_read(driver, query)

pprint(result)

[<Record br.name='Dois Corvos Cervejeira' b.name='Finisterra' nr_reviews=10>]


#### 6. On average how many different beer styles does each brewery produce?

In [14]:
query = """
        MATCH (b:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(s:STYLE)
        WITH b, count(distinct s) AS num_styles
        RETURN AVG(num_styles) AS avg_num_styles
    """

#We connected Breweries to Style through Beers
#We counted the distinct beer styles each brewery had in the "WITH"
#Then we returned the average

result = execute_read(driver, query)

pprint(result)

[<Record avg_num_styles=10.595992516144538>]


#### 7. Which brewery produces the strongest beers according to ABV?

In [15]:
query = """
        MATCH (bre:BREWERIES)-[:BREWED]->(b:BEERS)
        WHERE b.abv IS NOT NULL AND toFloat(b.abv)<100
        RETURN bre.name, AVG(toFloat(b.abv)) as abv_average
        ORDER BY abv_average desc
        LIMIT 1
    """

#We connected Breweries to Beers
#In the WHERE condition we made sure we weren't counting any beers which had no value in the abv using the IS NOT NULL
#We also made sure we weren't accounting beers which had abv equal to 100, since it would be just alchool 
#Then it was a matter of just returning the average abv value for the beers in each Brewery 


result = execute_read(driver, query)

pprint(result)

[<Record bre.name='Schorschbräu' abv_average=22.515384615384612>]


#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [16]:
query = """
        MATCH (s:STYLE)<-[:HAS_STYLE]-(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        WHERE r.smell<>"nan" AND r.look<>"nan" 
        RETURN s.name, AVG(toFloat(r.smell)+toFloat(r.look)) as Avg_smell_look_sum
        ORDER BY Avg_smell_look_sum desc
        LIMIT 1
    """
#We connected the beer Style to Reviews through Beer and using the "WHERE" condition we decide to not count the Reviews which 
#had missing values for at least one of the characteristics (smell and look).
#Then we added the values of these features, and then returned the average sum value for each beer style


result = execute_read(driver, query)

pprint(result)

[<Record s.name='New England IPA' Avg_smell_look_sum=8.797210377972341>]


#### 9. Using Graph Algorithms answer **one** of the following questions:
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them?
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community?

In [17]:
#2
# Step 0 - Clear graph, graph names need to be unique

try:
    query = """
            CALL gds.graph.drop('exercise_9') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `exercise_9` does not exist on database `neo4j`. It might exist on another database.')


In [18]:
try:
    query = """
        CALL gds.graph.project.cypher(
            'exercise_9',
            'MATCH (n:BEERS) RETURN id(n) AS id',
            'MATCH (b1:BEERS)-[:REVIEWED]-(r1:REVIEWS)-[:POSTED]-(u:USER)-[:POSTED]-(r2:REVIEWS)-[:REVIEWED]-(b2:BEERS) RETURN id(b1) AS source, id(b2) AS target limit 10000000'
        )
        """
    #Considering that beers are connected by users who reviewed them, we calculated which beer was connected to the 
    #highest number of other beers through users. This was our interpretion of influential
    
    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery='MATCH (n:BEERS) RETURN id(n) AS id' relationshipQuery='MATCH (b1:BEERS)-[:REVIEWED]-(r1:REVIEWS)-[:POSTED]-(u:USER)-[:POSTED]-(r2:REVIEWS)-[:REVIEWED]-(b2:BEERS) RETURN id(b1) AS source, id(b2) AS target limit 10000000' graphName='exercise_9' nodeCount=358873 relationshipCount=10000000 projectMillis=54713>]


In [19]:
# Step 2 - Run the algorithm

try:
    query = """
        CALL gds.pageRank.stream('exercise_9')
            YIELD nodeId, score
            RETURN gds.util.asNode(nodeId).name AS name, score
            ORDER BY score DESC
            limit 1
        """
    

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record name='Breakfast Stout' score=1.4330841235227758>]


#### 10. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    2. Which user is the most influential when it comes to reviews made?

In [24]:
# Step 0 - Clear graph

try:
    query = """
            CALL gds.graph.drop('exercise_10') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record graphName='exercise_10'>]


In [25]:
# Step 1 - Create an appropriate graph

try:
    query = """
        CALL gds.graph.project.cypher(
            'exercise_10',
            "MATCH (n) where head(Labels(n))='BEERS' or head(Labels(n))='REVIEWS' RETURN id(n) AS id",
            "MATCH (b1:BEERS)-[:REVIEWED]-(r1:REVIEWS)
                WITH b1, collect(r1.taste) as sbStoutTaste, collect(r1.feel) as sbStoutFeel
                WHERE b1.name = 'Super Bock Stout'

                MATCH (b2:BEERS)-[:REVIEWED]-(r2:REVIEWS)
                where any(x IN sbStoutTaste WHERE x = r2.taste)
                return id(b2) AS source, id(r2) AS target limit 1000"
   
            )
        """
    #We took as a criteria of similerity the taste and feel. For that we retrieve beers with the same taste and 
    #feel as SUPER BOCK STOUT

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery="MATCH (n) where head(Labels(n))='BEERS' or head(Labels(n))='REVIEWS' RETURN id(n) AS id" relationshipQuery="MATCH (b1:BEERS)-[:REVIEWED]-(r1:REVIEWS)\n                WITH b1, collect(r1.taste) as sbStoutTaste, collect(r1.feel) as sbStoutFeel\n                WHERE b1.name = 'Super Bock Stout'\n\n                MATCH (b2:BEERS)-[:REVIEWED]-(r2:REVIEWS)\n                where any(x IN sbStoutTaste WHERE x = r2.taste)\n                return id(b2) AS source, id(r2) AS target limit 1000" graphName='exercise_10' nodeCount=9431982 relationshipCount=1000 projectMillis=64749>]


In [26]:
# Step 2 - Run the algorithm

try:
    query = """
        CALL gds.nodeSimilarity.stream('exercise_10')
            YIELD node1, node2, similarity
            with gds.util.asNode(node1).name AS beer, gds.util.asNode(node2).name AS beer2, similarity
            WHERE beer >= 'Super Bock Stout'
            RETURN beer, beer2, similarity
            ORDER BY beer ASC, similarity desc
            limit 1
        """

    #We were unable to run the algorithm properly due to memory
    #We got 2 beers being one of them stout, acessed their similarity and returned the most similar one
    
    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[]


#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

In [23]:
query = """
        MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        WHERE toInteger(r.score) = 5 AND toInteger(r.overall) = 5
        RETURN b.name, count(r)
        ORDER BY count(r) DESC
        limit 3

    """

#On a first thought we tried to see the highest average on score and overall while at the same time a low standard deviation to have this
#as a proof of a consensual opinion. This did not reach a satisfactory result since the beers presented had a small number of reviews and 
#similar values.
#We believe that, even though their scores were high, there was not enough people to support a recommendation.
#Having this in mind, our next approach was to count, for every beer, the number of reviews which gave the highest value in both score and 
#overall. We selected only these 2 features because they evaluate the beer in a holistic way. From there, we selected the top 3 based on the 
#number of reviews

result = execute_read(driver, query)

pprint(result)

[<Record b.name='Heady Topper' count(r)=1024>,
 <Record b.name='Pliny The Elder' count(r)=727>,
 <Record b.name='KBS (Kentucky Breakfast Stout)' count(r)=569>]
